In [1]:
import pandas as pd

## 1) Definición de parámetros generales
- Archivo de fuentes de información
- Fecha de procesamiento

In [2]:
db = 'TableroInformacion.xlsx'
fecha_proceso = "2023-06-15"

## 2) Cargando datos
- Se carga los datos de las fuentes y flujos de los bonos
- Se elimina los espacios en blanco de las columnas

In [3]:
df_Bloomberg = pd.read_excel(db ,sheet_name='Bloomberg' , dtype = {'Fecha': str})
df_PIP = pd.read_excel(db ,sheet_name='PIP', dtype = {'Fecha': str})

df_SBS_PEN = pd.read_excel(db, sheet_name='SBS PEN', dtype = {'Fecha': str})
df_SBS_USD = pd.read_excel(db, sheet_name='SBS USD', dtype = {'Fecha': str})

df_Caracteristicas = pd.read_excel(db ,sheet_name='Caracteristicas')
df_Caracteristicas_variable = pd.read_excel(db ,sheet_name='Caracteristicas_variable')

df_flujos = pd.read_excel(db ,sheet_name='Flujo')

df_Bloomberg = df_Bloomberg.rename(columns=lambda x: x.strip())
df_PIP = df_PIP.rename(columns=lambda x: x.strip())
df_SBS_PEN = df_SBS_PEN.rename(columns=lambda x: x.strip())
df_SBS_USD = df_SBS_USD.rename(columns=lambda x: x.strip())
df_Caracteristicas = df_Caracteristicas.rename(columns=lambda x: x.strip())
df_flujos = df_flujos.rename(columns=lambda x: x.strip())


## 3) Limpieza de datos
- Eliminando espacios en blanco en la columna ISIN e Instrumento

In [4]:
df_Bloomberg['Instrumento'] = df_Bloomberg['Instrumento'].str.strip()
df_PIP['ISIN'] = df_PIP['ISIN'].str.strip()
df_SBS_PEN['ISIN'] = df_SBS_PEN['ISIN'].str.strip()
df_SBS_USD['ISIN'] = df_SBS_USD['ISIN'].str.strip()
df_Caracteristicas['ISIN'] = df_Caracteristicas['ISIN'].str.strip()
df_flujos['ISIN'] = df_flujos['ISIN'].str.strip()


- Formateando la columna Fecha

In [5]:
df_Bloomberg['Fecha'] = pd.to_datetime(df_Bloomberg['Fecha'], format='%Y-%m-%d')

df_PIP["Fecha"] = fecha_proceso
df_PIP['Fecha'] = pd.to_datetime(df_PIP['Fecha'], format='%Y-%m-%d')

df_SBS_PEN['Fecha'] = pd.to_datetime(df_SBS_PEN['Fecha'], format='%Y-%m-%d')
df_SBS_USD['Fecha'] = pd.to_datetime(df_SBS_USD['Fecha'], format='%Y-%m-%d')


- Homogeneizando valores de los precios

In [6]:
#igualando a la fecha de proceso y quitando lo de bloomberg que pertebece a la renta variable
df_Bloomberg_rent_fijo = df_Bloomberg[(df_Bloomberg.Fuente!="BLG") & (df_Bloomberg.Fecha==fecha_proceso) ].copy()

#igualando a la fecha de proceso y quitando lo de bloomberg que pertebece a la renta variable
df_Bloomberg_rent_variable = df_Bloomberg[(df_Bloomberg.Fuente=="BLG") & (df_Bloomberg.Fecha==fecha_proceso) ].copy()


df_Bloomberg_rent_fijo["Valor"] = df_Bloomberg_rent_fijo["Valor"]/100

df_PIP["Fuente"] = "PIP"

df_SBS_PEN["Fuente"] = "SBS"
df_SBS_PEN["P. Limpio (%)"] = df_SBS_PEN["P. Limpio (%)"]/100

df_SBS_USD["Fuente"] = "SBS"
df_SBS_USD["P. Limpio (%)"] = df_SBS_USD["P. Limpio (%)"]/100

## 4) Renta Variable
- Generación del archivo de vector de precios de renta variable

In [7]:

df_rent_variable = df_Bloomberg_rent_variable.copy()

#Ticker	Emisor	Moneda	Precio	Fuente

df_rent_variable.rename(columns={"Valor": "Precio","Instrumento": "Ticker"},inplace=True)

df_rent_variable = df_rent_variable[["Fecha",'Ticker',"Precio","Fuente"]].copy()

df_rent_variable = pd.merge(df_rent_variable,df_Caracteristicas_variable,right_on="Ticker",left_on="Ticker",how="inner")

df_rent_variable = df_rent_variable[["Ticker","Emisor","Moneda","Precio","Fuente"]].copy()
df_rent_variable

,Ticker,Emisor,Moneda,Precio,Fuente
0,5HED LN,Ossiam ESG Low Carbon Shiller Barclays Cape US...,USD,114.65,BLG
1,AAL LN,Anglo American PLC,GBp,2580.00,BLG
2,AAL US,American Airlines Group Inc,USD,16.65,BLG
3,AAPL US,Apple Inc,USD,186.01,BLG
4,AAU CN,Angold Resources Ltd,CAD,0.08,BLG
5,AAXJ US,iShares MSCI All Country Asia ex Japan ETF,USD,69.13,BLG
6,ABBV US,AbbVie Inc,USD,135.89,BLG
7,ABEV US,Ambev SA,USD,3.13,BLG


## 5) Renta Fija

- Generación del dataframe temporal "df_input_precio_bono_tmp" que centraliza los precios de las distintas fuentes. Este dataframe es la base para poder generar el dataframe "df_precio".

In [8]:
ipb_cls =  ['Fecha', 'ISIN', 'Precio','Fuente']

df_input_precio_bono_tmp = pd.DataFrame(columns = ipb_cls)

df_temp=pd.DataFrame(df_Bloomberg_rent_fijo[["Fecha","Instrumento","Valor","Fuente"]].values, columns=ipb_cls)
df_input_precio_bono_tmp = df_input_precio_bono_tmp.append(df_temp)

df_temp=pd.DataFrame(df_PIP[["Fecha","ISIN","Precio Limpio %","Fuente"]].values, columns=ipb_cls)
df_input_precio_bono_tmp = df_input_precio_bono_tmp.append(df_temp)

df_temp=pd.DataFrame(df_SBS_PEN[["Fecha","ISIN","P. Limpio (%)","Fuente"]].values, columns=ipb_cls)
df_input_precio_bono_tmp = df_input_precio_bono_tmp.append(df_temp)

df_temp=pd.DataFrame(df_SBS_USD[["Fecha","ISIN","P. Limpio (%)","Fuente"]].values, columns=ipb_cls)
df_input_precio_bono_tmp = df_input_precio_bono_tmp.append(df_temp)

df_input_precio_bono_tmp.head()

,Fecha,ISIN,Precio,Fuente
0,2023-06-15,US912810EQ77,1.000664,TRAC
1,2023-06-15,US912810EQ77,1.000664,BVAL
2,2023-06-15,US912810EQ77,0.998906,BGN
3,2023-06-15,US912810FJ26,1.119219,BGN
4,2023-06-15,US912810FJ26,1.119297,BVAL


- Desarrollo de funciones utilitarias para el vector de precios de renta fija
    - set_metodo_nuevo: Calculo de la variable "MetodoValNuevo" que define el método de cálculo que se empleara para el precio del activo
    - set_precio_nuevo: Determina el "PrecioLimpio" correspondiente a la variable "MetodoValNuevo"
    - calculo_TIR: Cálculo del TIR
    - get_TIR_anterior: Calculo del TIR anterior

In [9]:
def set_metodo_nuevo(fila):

    if fila['ISIN'][:3]=="PEP" :

        if fila['Tipo Bono']=="Certificado Deposito":  
        # Aplicar la lógica del switch case
            if pd.isna(fila['PIP'])==False:
                return 'PIP'
            else:
                return 'TIR'
        
        else:

            if pd.isna(fila['PIP'])==False:
                return 'PIP'
            elif pd.isna(fila['SBS'])==False:
                return 'SBS'
            else:
                return 'TIR'          
        
    else:

        if pd.isna(fila['BVAL'])==False:
            return 'BVAL'
        if pd.isna(fila['BGN'])==False:
            return 'BGN'
        if pd.isna(fila['TRAC'])==False:
            return 'TRAC'
        else:
            return 'TIR'
        

def set_precio_nuevo(fila):

    MetodoValNuevo = fila['MetodoValNuevo']

    return fila[MetodoValNuevo]

def calculo_TIR(fila):

    tir = ((fila['Amortización'])/((1+fila['Tasa Bono'])**(fila['Plazo']/365)))

    return tir


def get_TIR_anterior(fecha_proceso_param,df_flujos_,isin):

    fecha_proceso_dt = pd.to_datetime(fecha_proceso_param)

    df_flujos_bono = df_flujos_[(df_flujos_.ISIN==isin) & (df_flujos_["Fecha Vto"]<fecha_proceso_dt)].copy()

    df_flujos_bono.sort_values(by="Fecha Vto",ascending=True,inplace=True)

    df_flujos_bono["Plazo"] =  (df_flujos_bono["Fecha Vto"] - fecha_proceso_dt).dt.days

   
    df_flujos_bono['TIR'] = df_flujos_bono.apply(calculo_TIR, axis=1)

    df_flujos_bono_agg = df_flujos_bono.groupby(["ISIN"],as_index=False).agg({'TIR': "sum"})
    df_flujos_bono_agg["TIR"] = df_flujos_bono_agg["TIR"]/100


    TIR = df_flujos_bono_agg.loc[0, 'TIR'] 

    return TIR

- Generación del dataframe utilitario "df_precio" que lista los precios de los activos de renta fija 

In [10]:
cls_carac = ["ISIN","Pais Emisor","Tipo Bono"]
df_precio = df_Caracteristicas[cls_carac].copy()
print(df_precio.shape)
for fuente in ["PIP","BVAL","BGN","TRAC","SBS","TIR"]:
    df_ipb_temp= df_input_precio_bono_tmp[(df_input_precio_bono_tmp.Fecha==fecha_proceso) & (df_input_precio_bono_tmp.Fuente==fuente)][["ISIN","Precio"]].copy()
    df_ipb_temp.rename(columns={"Precio": fuente},  inplace=True )
    df_precio = pd.merge(df_precio,df_ipb_temp,left_on="ISIN",right_on="ISIN",how="left")

    if fuente == "TIR":
        df_precio['TIR'] = None
        for index, row in df_precio.iterrows():
            # Obtén el valor actual de 'columna_actualizar'
            isin = row['ISIN']
            try:
                tir = get_TIR_anterior(fecha_proceso,df_flujos,isin)
                # Actualiza el valor en la columna correspondiente     
                df_precio.at[index, 'TIR'] = tir
            except:
                continue

df_precio['MetodoValNuevo'] = df_precio.apply(set_metodo_nuevo, axis=1)
df_precio['PrecioLimpio'] = df_precio.apply(set_precio_nuevo, axis=1)

df_precio

(31, 3)


,ISIN,Pais Emisor,Tipo Bono,PIP,BVAL,BGN,TRAC,SBS,TIR,MetodoValNuevo,PrecioLimpio
0,PEP12100D243,PE,Bono Corporativo,NaN,NaN,NaN,NaN,1.01315,None,SBS,1.013150
1,PEP58501M073,PE,Bono Soberano,0.991568,NaN,NaN,NaN,0.96688,None,PIP,0.991568
2,PEP72760M010,PE,Bono Corporativo,0.91496,NaN,NaN,NaN,NaN,None,PIP,0.914960
3,PEP73125M049,US,Bono Corporativo,0.987375,NaN,NaN,NaN,NaN,None,PIP,0.987375
4,PEP73125M056,US,Bono Corporativo,0.964907,NaN,NaN,NaN,NaN,None,PIP,0.964907
5,PEP73125M064,PE,Bono Soberano,0.901541,NaN,NaN,NaN,NaN,None,PIP,0.901541
6,US912810EQ77,US,Bono Tesoro,NaN,1.000664,0.998906,1.000664,NaN,0.0,BVAL,1.000664
7,US912810FJ26,US,Bono Tesoro,NaN,1.119297,1.119219,1.119297,NaN,None,BVAL,1.119297
8,US912810FT08,US,Bono Tesoro,NaN,1.085625,1.084844,1.085625,NaN,0.0,BVAL,1.085625
9,US912810PU60,US,Bono Tesoro,NaN,1.13875,1.137969,1.13875,NaN,0.0,BVAL,1.138750


- Desarrollo de funciones utilitarias para el desarrollo de "df_renta_fija"
    - calculo_duracion: Permite calcular la duración
    - get_plazo_corrido: permite poner el plazo corrido
    - get_duracion_and_TIR: Calcula la duración y TIR

In [11]:

def calculo_duracion(fila):
    try:
        duracion = ((fila['Amortización']*fila['Plazo']/365)/((1+fila['Tasa Bono'])**(fila['Plazo']/365)))/(fila['Amortización']/(1+fila['Tasa Bono'])**(fila['Plazo']/365))
    except:
        duracion = 0

    return duracion



def get_plazo_corrido(fecha_proceso,base_calculo, df_flujos_,isin):

    fecha_proceso_dt = pd.to_datetime(fecha_proceso)

    df_flujos_bono = df_flujos_[(df_flujos_.ISIN==isin) & (df_flujos_["Fecha Vto"]<fecha_proceso_dt)].copy()

    if len(df_flujos_bono)==0:
        return None

    df_flujos_bono.sort_values(by="Fecha Vto",ascending=True,inplace=True)

    FecVtoAnteriorCupo = df_flujos_bono['Fecha Vto'].iloc[-1]

    if base_calculo=="ACT/ACT":

        plazoCorrido =  (fecha_proceso_dt - FecVtoAnteriorCupo).days

    else:

        D1 = FecVtoAnteriorCupo.day
        M1 = FecVtoAnteriorCupo.month
        Y1 = FecVtoAnteriorCupo.year

        D2 = fecha_proceso_dt.day
        M2 = fecha_proceso_dt.month
        Y2 = fecha_proceso_dt.year

        if base_calculo=="030/360":
            DIAS_YEAR = 360
            DIAS_MONTH = 30
        if base_calculo=="030/365":
            DIAS_YEAR = 365
            DIAS_MONTH = 30
        if base_calculo=="ACT/360":
            DIAS_YEAR = 360
            DIAS_MONTH = fecha_proceso_dt.days_in_month


        plazoCorrido = (Y2-Y1)*DIAS_YEAR + (M2-M1)*DIAS_MONTH + (D2-D1)    

    return plazoCorrido

def get_duracion_and_TIR(fecha_proceso_param,df_flujos_param,isin):

    fecha_proceso_dt = pd.to_datetime(fecha_proceso_param)

    df_flujos_bono = df_flujos_param[(df_flujos_param.ISIN==isin) & (df_flujos_param["Fecha Vto"]>=fecha_proceso_dt)].copy()

    if len(df_flujos_bono)==0:
        return None , None

    df_flujos_bono.sort_values(by="Fecha Vto",ascending=True,inplace=True)

    df_flujos_bono["Plazo"] =  (df_flujos_bono["Fecha Vto"] - fecha_proceso_dt).dt.days

    df_flujos_bono['Duracion'] = df_flujos_bono.apply(calculo_duracion, axis=1)
    df_flujos_bono['TIR'] = df_flujos_bono.apply(calculo_TIR, axis=1)

    df_flujos_bono_agg = df_flujos_bono.groupby(["ISIN"],as_index=False).agg({'Duracion': "sum", 'TIR': "sum"})
    df_flujos_bono_agg["TIR"] = df_flujos_bono_agg["TIR"]/100

    Duracion = df_flujos_bono_agg.loc[0, 'Duracion']  # Valor en la primera fila, columna1
    TIR = df_flujos_bono_agg.loc[0, 'TIR'] 

    return Duracion , TIR





Generación del dataframe "df_renta_fija"

In [12]:
df_renta_fija = df_Caracteristicas.copy()

#Fecha	ISIN	Emisor	Moneda	Tasa Cupon	Fec. Vcto	TIR Mcdo	Precio Limpio (%)	Int. Corrido (%)	Reajuste VAC	Precio Mcdo (%)	Dur. Normal	Fuente

for index, row in df_renta_fija.iterrows():
    #try:
    # Obtén el valor actual de 'columna_actualizar'
    isin = row['ISIN']
    base_calculo = row['Base Calculo Interes Corrido']
    #print(f"-{isin}-")
    dur, tir = get_duracion_and_TIR(fecha_proceso,df_flujos,isin)
    plazoCorrido = get_plazo_corrido(fecha_proceso,base_calculo,df_flujos,isin)
    # Actualiza el valor en la columna correspondiente
    df_renta_fija.at[index, 'PlazoCorrido'] = plazoCorrido


    df_renta_fija.at[index, 'Duracion'] = dur
    df_renta_fija.at[index, 'TIR'] = tir
    #except:
    #    continue

df_renta_fija["InteresCorrido"] = df_renta_fija["Tasa cupon"]*1*df_renta_fija["PlazoCorrido"] / 365

df_renta_fija = pd.merge(df_renta_fija,df_precio[["ISIN","MetodoValNuevo","PrecioLimpio"]],right_on="ISIN",left_on="ISIN",how="left")

df_renta_fija["PrecioSucio"] = df_renta_fija["InteresCorrido"]  + df_renta_fija["InteresCorrido"]

df_renta_fija["Reajuste VAC"]  = None
df_renta_fija["Precio Mcdo (%)"]  = None

df_renta_fija


,ISIN,Fecha Inicio,Fecha Vto,Tasa cupon,Frecuencia Pago,Base Calculo Interes Corrido,Tipo Tasa,Emisor,Pais Emisor,Moneda,Tipo Bono,PlazoCorrido,Duracion,TIR,InteresCorrido,MetodoValNuevo,PrecioLimpio,PrecioSucio,Reajuste VAC,Precio Mcdo (%)
0,PEP12100D243,2016-04-06,2024-04-06,0.000000,S,ACT/ACT,Variable,BANCO INTERAMERICANO DE FINANZAS,PE,USD,Bono Corporativo,NaN,NaN,NaN,NaN,SBS,1.013150,NaN,None,None
1,PEP58501M073,2018-11-14,2033-11-14,0.068400,S,ACT/ACT,Fijo,GOBIERNO DEL PERU,PE,PEN,Bono Soberano,NaN,NaN,NaN,NaN,PIP,0.991568,NaN,None,None
2,PEP72760M010,2022-02-25,2037-02-25,0.072813,S,030/360,Fijo,CENTENARIO RENTA INMOBILIARIA S.A.C.,PE,USD,Bono Corporativo,NaN,NaN,NaN,NaN,PIP,0.914960,NaN,None,None
3,PEP73125M049,2018-07-23,2028-07-23,0.070625,S,030/360,Fijo,CORPORACIÓN PRIMAX SA,US,PEN,Bono Corporativo,NaN,NaN,NaN,NaN,PIP,0.987375,NaN,None,None
4,PEP73125M056,2018-07-23,2038-07-23,0.076875,S,030/360,Fijo,CORPORACIÓN PRIMAX SA,US,PEN,Bono Corporativo,NaN,NaN,NaN,NaN,PIP,0.964907,NaN,None,None
5,PEP73125M064,2019-11-15,2031-11-15,0.059687,S,ACT/360,Fijo,GOBIERNO DEL PERU,PE,PEN,Bono Soberano,NaN,NaN,NaN,NaN,PIP,0.901541,NaN,None,None
6,US912810EQ77,1993-08-16,2023-08-16,0.062500,S,ACT/ACT,Fijo,US TREASURY N/B,US,USD,Bono Tesoro,119.0,0.169863,0.989755,0.020377,BVAL,1.000664,0.040753,None,None
7,US912810FJ26,1999-08-16,2029-08-15,0.061250,S,ACT/ACT,Fijo,US TREASURY,US,USD,Bono Tesoro,NaN,NaN,NaN,NaN,BVAL,1.119297,NaN,None,None
8,US912810FT08,2006-02-15,2036-02-15,0.045000,S,ACT/ACT,Fijo,US TREASURY N/B,US,USD,Bono Tesoro,120.0,12.679452,0.572290,0.014795,BVAL,1.085625,0.029589,None,None
9,US912810PU60,2007-08-15,2037-05-15,0.050000,S,ACT/ACT,Fijo,US TREASURY,US,USD,Bono Tesoro,31.0,13.926027,0.506894,0.004247,BVAL,1.138750,0.008493,None,None


## 6) Controles de validación

TIR: La rentabilidad del instrumento deberá ser positiva

In [13]:
#df_renta_fija = df_renta_fija[(df_renta_fija["TIR"] >= 0)].copy()

Solo deben considerarse instrumentos de renta fija presente fecha de próximo
cupón o vencimiento.

In [14]:
fecha_proceso_dt = pd.to_datetime(fecha_proceso)
df_renta_fija = df_renta_fija[(df_renta_fija["Fecha Vto"] >= fecha_proceso_dt)].copy()
#df_renta_fija

## 7) Exportación

- Formateando el dataframe con las variables solicitadas

In [15]:
cls_ori = ["Fecha Vto","ISIN","Emisor","Moneda","Tasa cupon","Fecha Vto","TIR","PrecioLimpio","InteresCorrido","Reajuste VAC","PrecioLimpio","Duracion","MetodoValNuevo"] 
cls_des = ["Fecha","ISIN",	"Emisor","Moneda","Tasa Cupon","Fec. Vcto","TIR Mcdo","Precio Limpio (%)","Int. Corrido (%)","Reajuste VAC","Precio Mcdo (%)","Dur. Normal","Fuente"] 

df_renta_fija=pd.DataFrame(df_renta_fija[cls_ori].values, columns=cls_des)



In [16]:
df_renta_fija

,Fecha,ISIN,Emisor,Moneda,Tasa Cupon,Fec. Vcto,TIR Mcdo,Precio Limpio (%),Int. Corrido (%),Reajuste VAC,Precio Mcdo (%),Dur. Normal,Fuente
0,2024-04-06,PEP12100D243,BANCO INTERAMERICANO DE FINANZAS,USD,0.0,2024-04-06,NaN,1.01315,NaN,None,1.01315,NaN,SBS
1,2033-11-14,PEP58501M073,GOBIERNO DEL PERU,PEN,0.0684,2033-11-14,NaN,0.991568,NaN,None,0.991568,NaN,PIP
2,2037-02-25,PEP72760M010,CENTENARIO RENTA INMOBILIARIA S.A.C.,USD,0.072813,2037-02-25,NaN,0.91496,NaN,None,0.91496,NaN,PIP
3,2028-07-23,PEP73125M049,CORPORACIÓN PRIMAX SA,PEN,0.070625,2028-07-23,NaN,0.987375,NaN,None,0.987375,NaN,PIP
4,2038-07-23,PEP73125M056,CORPORACIÓN PRIMAX SA,PEN,0.076875,2038-07-23,NaN,0.964907,NaN,None,0.964907,NaN,PIP
5,2031-11-15,PEP73125M064,GOBIERNO DEL PERU,PEN,0.059687,2031-11-15,NaN,0.901541,NaN,None,0.901541,NaN,PIP
6,2023-08-16,US912810EQ77,US TREASURY N/B,USD,0.0625,2023-08-16,0.989755,1.000664,0.020377,None,1.000664,0.169863,BVAL
7,2029-08-15,US912810FJ26,US TREASURY,USD,0.06125,2029-08-15,NaN,1.119297,NaN,None,1.119297,NaN,BVAL
8,2036-02-15,US912810FT08,US TREASURY N/B,USD,0.045,2036-02-15,0.57229,1.085625,0.014795,None,1.085625,12.679452,BVAL
9,2037-05-15,US912810PU60,US TREASURY,USD,0.05,2037-05-15,0.506894,1.13875,0.004247,None,1.13875,13.926027,BVAL


- Formateando el dataframe con las variables solicitadas

In [17]:

with pd.ExcelWriter('EstructuraVectorFinal.xlsx') as writer:      
    df_renta_fija.to_excel(writer, sheet_name='RentaFija')
    df_rent_variable.to_excel(writer, sheet_name='RentaVariable')